[View in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/index.ipynb)

# Combinación de bases de datos de artículos científicos
```
URL_base=https://colab.research.google.com/github/blob_file_in_repo
blob_file_in_repo → restrepo/medicion/blob/master/cienciometria/merge.ipynb

```
## UdeA
### Combinar bases de datos bibliográficas UDEA
* [Combibnar W0S+..](https://colab.research.google.com/merge.ipynb)